In [1]:
%pylab inline
%config InlineBackend.figure_format = 'retina'

Populating the interactive namespace from numpy and matplotlib


In [2]:
import os

import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt
from matplotlib import cm

from ficus import FigureManager
from IPython.display import FileLink

import yaml

In [3]:
with open('config.yaml') as fp:
    info = yaml.load(fp)

In [4]:
sns.set(style="ticks")

In [28]:
samples.head()
subsamples = [ t for t in list(samples.columns.unique()) if int(t) % 1000000 == 0 ]

In [30]:
samples[subsamples]

,1000000,2000000,3000000,4000000,5000000,6000000,7000000,8000000,9000000,10000000,...,26000000,27000000,28000000,29000000,30000000,31000000,32000000,33000000,34000000,35000000
0,1,1,1,4,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,1,1,1,1,1,7,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1,1,1,1,...,4,1,1,1,1,1,1,4,1,1
3,1,1,1,13,1,1,1,1,1,1,...,4,1,1,1,1,1,1,1,4,1
4,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,4,1,4,1,1
5,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,12,1,1,1,4,1
6,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,4
7,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,10,1,1,1
8,1,1,1,1,1,1,1,1,1,1,...,1,4,1,1,1,1,1,1,1,1
9,1,1,1,4,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [31]:
def plot_component_size_samples(directory, interval=1000000, ax=None):
    import ast

    data = pd.read_csv('{0}/boink.cdbg.components.csv'.format(directory),
                       converters={'component_size_sample': lambda col: ast.literal_eval(col)})
    data = data[:-1]
    samples = pd.DataFrame.from_items(zip(data.read_n, data.component_size_sample.values))
    
    def _plot(ax):
        subsamples = [ t for t in list(samples.columns.unique()) if int(t) % interval == 0 ]
        _samples = samples[subsamples]
        _samples = _samples.apply(lambda x: x.sort_values().values)
        cums = _samples.cumsum()
        sums = _samples.sum()
        (cums / sums).plot(ax=ax, colormap = cm.autumn_r)
        #ax.set_title('Cumulative Component Size Proportion')   
    
    if ax is not None:
        _plot(ax)
    else:
        with FigureManager('{0}/component_sizes_cumsum.png'.format(directory),
                           figsize=(12,12),
                           show=True) as (fig, ax):
            _plot(ax)
    return samples

In [ ]:
with FigureManager(figsize=(18,18), show=True, nrows=len(info['samples']), ncols=3) as (fig, ax):
    for n, sample_name in enumerate(info['samples']):
        for m, ksize in enumerate(info['samples'][sample_name]['ksizes']):
            directory  = os.path.join('outputs', sample_name, 'component_sampling', str(ksize))
            stats_file = os.path.join(directory, 'boink.cdbg.components.csv')
            name       = '{0}, K={1}'.format(' '.join(sample_name.split('_')), ksize)

            try:
                samples = plot_component_size_samples(directory, ax=ax[n][m])
            except:
                print('Compute not finished yet.')
            ax[n][m].set_title('{0}, K={1}'.format(sample_name, ksize))

/home/camille/miniconda/envs/libboink/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  import sys
